In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
# droping nan values
df = df.dropna()

In [6]:
X = df.drop('label', axis=1)

In [7]:
y = df['label']

In [8]:
X.shape

(18285, 4)

In [10]:
import tensorflow as tf

In [11]:
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential

In [12]:
voc_size=5000

Onehot Representation

In [13]:
messages = X.copy()

In [14]:
messages.reset_index(inplace=True)

In [16]:
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem import PorterStemmer

In [28]:
# Data Preprossing 
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in set(stopwords.words('english'))]
    review = " ".join(review)
    corpus.append(review)

In [29]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [31]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[3947, 539, 4054, 3251, 4208, 336, 2236, 4298, 2636, 1265],
 [2641, 3289, 3098, 2951, 4625, 4153, 889],
 [3460, 1062, 2200, 3300],
 [4256, 3597, 320, 307, 4184, 2177],
 [3414, 4625, 4968, 415, 531, 544, 4625, 4364, 4538, 3786],
 [1151,
  3066,
  1562,
  2864,
  2329,
  2649,
  2399,
  1003,
  4850,
  1847,
  2338,
  635,
  1956,
  855,
  889],
 [1546, 18, 1578, 561, 2510, 501, 975, 2168, 2746, 3960, 2482],
 [3761, 4367, 1701, 797, 1706, 3052, 2649, 785, 2746, 3960, 2482],
 [2116, 3892, 4510, 245, 725, 3040, 3017, 1751, 2649, 2249],
 [1572, 1650, 3196, 1017, 918, 4712, 1, 3031],
 [2644, 3499, 683, 1141, 509, 1760, 4678, 107, 3310, 523, 16],
 [307, 141, 4208, 3040, 2649, 1706],
 [4620, 4236, 635, 344, 845, 1968, 2036, 3866, 2791],
 [4001, 2948, 529, 822, 1852, 1532, 3675, 2746, 3960, 2482],
 [628, 922, 254, 2236, 3529, 2746, 3960, 2482],
 [4620, 4879, 266, 807, 1328, 2884, 3981, 4365, 3215, 4311],
 [4072, 4994, 3289],
 [3749, 1456, 4153, 2622, 2649, 4390, 885, 889],
 [1820, 4190, 3098, 

Embedding Representation

In [32]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 4298, 2636, 1265],
       [   0,    0,    0, ..., 4625, 4153,  889],
       [   0,    0,    0, ..., 1062, 2200, 3300],
       ...,
       [   0,    0,    0, ..., 2746, 3960, 2482],
       [   0,    0,    0, ..., 2594, 4775, 2901],
       [   0,    0,    0, ..., 2606, 3745, 2416]])

In [33]:
vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, vector_features, input_length=sent_length))
# in embedding layer first feature is vocabilary size and second is feature size and sentence length
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [34]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [35]:
X_final.shape

(18285, 20)

In [36]:
y_final.shape

(18285,)

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

Model Training

In [39]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 31ms/step - loss: 0.3304 - accuracy: 0.8433 - val_loss: 0.1894 - val_accuracy: 0.9206
Epoch 2/10
192/192 [==============================] - 6s 33ms/step - loss: 0.1400 - accuracy: 0.9476 - val_loss: 0.1881 - val_accuracy: 0.9223
Epoch 3/10
192/192 [==============================] - 5s 27ms/step - loss: 0.0932 - accuracy: 0.9656 - val_loss: 0.2405 - val_accuracy: 0.9193
Epoch 4/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0625 - accuracy: 0.9793 - val_loss: 0.2540 - val_accuracy: 0.9191
Epoch 5/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0377 - accuracy: 0.9881 - val_loss: 0.2766 - val_accuracy: 0.9181
Epoch 6/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0228 - accuracy: 0.9926 - val_loss: 0.3440 - val_accuracy: 0.9157
Epoch 7/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0146 - accuracy: 0.9959 - val_loss: 0.4071 - val_accuracy: 0.9155

Adding Dropout

In [62]:
from tensorflow.keras.layers import Dropout
new_model = Sequential()
new_model.add(Embedding(voc_size, vector_features, input_length=sent_length))
new_model.add(Dropout(0.3))
new_model.add(LSTM(100))
new_model.add(Dropout(0.3))
new_model.add(Dense(1, activation='sigmoid'))
new_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Performance Metrics And Accuracy

In [57]:
y_pred=model.predict(X_test)

In [58]:
y_pred

array([[9.9998927e-01],
       [3.0881739e-07],
       [2.4070068e-05],
       ...,
       [3.9236406e-07],
       [9.9833912e-01],
       [8.3051926e-01]], dtype=float32)

In [48]:
y_pred1 = np.argmax(y_pred,axis=1)
y_pred1

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [55]:
from sklearn.metrics import confusion_matrix

In [56]:
confusion_matrix(y_test,y_pred1)

array([[3419,    0],
       [2616,    0]], dtype=int64)

In [60]:

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.5665285832642917